TODO: 
    
    - separar en carga raw, manipulacion, eda
    - no hacer ese fill na cuando paso bool a int (astype?)
    - sacar las listas temporales de cols a elegir y quedarme con cols

In [ ]:
# TODO: checkear y cambiar tipos antes de persistir df final?
# cambiar ahi el boolean a int de las cols que quiero,!!!

In [1]:
import json
from urllib.request import urlopen
import urllib
import pandas as pd
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
import pickle 

In [2]:
os.chdir("..") 

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt { font-size: 08px !important; }</style>"))
pd.options.display.max_columns = None


In [4]:
data_path = "Data/"

In [5]:
pkl_path = "pkl/"

# 1️⃣ Dataload 
---

In [6]:
url_base = "https://api.mercadolibre.com/"

In [7]:
url_MLA = url_base + "/sites/MLA/"

In [8]:
limit = 50 # fixed

In [9]:
items_per_category = 1000 # chosen

In [10]:
requests_per_category = int(items_per_category / limit)
requests_per_category

20

## Fxs

In [11]:
def get_data(url):
    # makes a get request given a url, and returns json response
    response = urllib.request.urlopen(url)
    return json.load(response)
    

In [12]:
def load_category_items(categ_id):
    # gets items given a category id
    url_categ = url_MLA + "search?&category={}".format(categ_id)
    all_results = [] # offset and limit must be taken into account to reach items_per_category
    for r in range(0, requests_per_category):
        offset = r * limit
        url = url_categ + "&offset={}".format(offset)
        response = get_data(url)
        results = response['results'] # keep only the items
        all_results = all_results + results
    return all_results 
    

Test

all_results = load_category_items("MLA1367")


len(all_results)

## Get root categories

In [13]:
url = url_MLA + "categories"

In [14]:
root_categories = get_data(url)
root_categories

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas y Construcción'},
 {'id': 'MLA1574', 'name': 'Hogar, Muebles y Jardín'},
 {'

## Get items per root category

In [15]:
df = pd.DataFrame() # will contain items from all root categories
for c in tqdm(root_categories):
    all_results = load_category_items(c["id"])
    df_tmp = pd.json_normalize(all_results) # build a pd df from a json with nested data (notice how lists remain lists)
    # add root category data to the df with category items
    df_tmp["root_category_id"] = c["id"]
    df_tmp["root_category_name"] = c["name"]
    # append df_tmp to the final df
    df = df.append(df_tmp, ignore_index=True)

  0%|          | 0/31 [00:00<?, ?it/s]

**Persist raw data**

In [16]:
file_name = data_path + "raw_data.csv"

In [17]:
df.to_csv(file_name, sep='\t', encoding='utf-8', index=False)

**Inspect df**

In [18]:
df.shape

(30925, 132)

In [19]:
df.columns.tolist()

['id',
 'site_id',
 'title',
 'price',
 'sale_price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'buying_mode',
 'listing_type_id',
 'stop_time',
 'condition',
 'permalink',
 'thumbnail',
 'thumbnail_id',
 'accepts_mercadopago',
 'attributes',
 'original_price',
 'category_id',
 'official_store_id',
 'domain_id',
 'catalog_product_id',
 'tags',
 'catalog_listing',
 'use_thumbnail_id',
 'order_backend',
 'seller.id',
 'seller.permalink',
 'seller.registration_date',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'seller.tags',
 'seller.eshop.nick_name',
 'seller.eshop.eshop_rubro',
 'seller.eshop.eshop_id',
 'seller.eshop.eshop_locations',
 'seller.eshop.site_id',
 'seller.eshop.eshop_logo_url',
 'seller.eshop.eshop_status_id',
 'seller.eshop.seller',
 'seller.eshop.eshop_experience',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.transacti

Search for specific columns 

In [20]:
[x for x in df.columns if "installments" in x]

['installments.quantity',
 'installments.amount',
 'installments.rate',
 'installments.currency_id',
 'installments']

In [21]:
df['installments.rate'].value_counts()

70.17    26093
0.00      2581
Name: installments.rate, dtype: int64

In [22]:
df.category_id[0]

'MLA6867'

## Get subcategories info
Recursive algorithm

## TODO: metodo recursivo para ir pidiendo las subcategorias, asi le mapeo la categoria padre y la específica a cada item

- no se si sirve hacer una lista de todo el arbol de categorias????? -- emb?
- o bien mapear la categ específica a cada row ..... para aperturar el análisis que haga, y no quedarme con lo que veo a nivel root category nada más

In [23]:
def get_subcategories_from_root(categ_id, dic):
    # recursive function to get subcategories data 
    url = url_base + "categories/{}".format(categ_id)
    response = get_data(url)
    children_categories = response["children_categories"]
    for cc in children_categories: # base case: children_categories list is empty
        id = cc["id"]
        dic[id] = {"name": cc["name"], 
                    "root": response["path_from_root"]} # 'root' does not include current category as response["path_from_root"] has parent category data 
        get_subcategories_from_root(id, dic)  

Test 

In [24]:
dic = {}
res = get_subcategories_from_root("MLA4711", dic) # INFO: takes 2 minutes

In [25]:
dic

{'MLA86379': {'name': 'Alarmas para Motos',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'}]},
 'MLA434708': {'name': 'Alforjas',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'}]},
 'MLA11090': {'name': 'Baúles y Anclajes',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'}]},
 'MLA86350': {'name': 'Baúles Laterales',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'},
   {'id': 'MLA11090', 'name': 'Baúles y Anclajes'}]},
 'MLA432921': {'name': 'Baúles para Cuatriciclos',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'},
   {'id': 'MLA11090', 'name': 'Baúles y Anclajes'}]},
 'MLA379646': {'

**Get subcategories info for all root categories**

In [26]:
# FIXME: takes a lot of time ... maybe just get subcategies from selected group of root categories instead of all

In [27]:
dic_all_categories = {}
for c in tqdm(root_categories):
    dic = {}
    get_subcategories_from_root(c["id"], dic)  
    dic_all_categories.update(dic)

  0%|          | 0/31 [00:00<?, ?it/s]

**Inspect dic_all_categories**

In [28]:
len(dic_all_categories)

9545

In [29]:
first_key = next(iter(dic_all_categories)) 
first_key

'MLA4711'

In [30]:
dic_all_categories[first_key]

{'name': 'Acc. para Motos y Cuatriciclos',
 'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'}]}

**Persist dic_all_categories**

In [31]:
file_name = pkl_path + "dic_all_categories.pkl"

In [32]:
with open(file_name, 'wb') as handle:
    pickle.dump(dic_all_categories, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 2️⃣ Manipulate df
---

In [33]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.car_dealer_logo,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms
0,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,None,ARS,4,19,buy_it_now,gold_special,2040-12-17T04:00:00.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'attribute_group_name': 'Otros', 'name': 'Ma...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[loyalty_discount_eligible, brand_verified, ex...",True,True,1,8911722,http://perfil.mercadolibre.com.ar/AR+ESCALADA,2011-04-19

## Add columns

**Add discount column**

In [34]:
df["discount"] = 1 - df.price / df.original_price if df.original_price is not None else None

In [35]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.car_dealer_logo,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms,discount
0,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,None,ARS,4,19,buy_it_now,gold_special,2040-12-17T04:00:00.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'attribute_group_name': 'Otros', 'name': 'Ma...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[loyalty_discount_eligible, brand_verified, ex...",True,True,1,8911722,http://perfil.mercadolibre.com.ar/AR+ESCALADA,2

Inspect discount coulumn

In [36]:
df[~df.original_price.isna()][["id", "site_id", "title", "price", "sale_price", "original_price", "discount"]]

,id,site_id,title,price,sale_price,original_price,discount
2,MLA862644604,MLA,Casco Para Moto Integral Halcon H57 Negro Tall...,2289.50,None,2410,0.05
3,MLA836881964,MLA,Compresor De Aire Alto Impacto Portátil 250 Ps...,1899.00,None,2799,0.321543
8,MLA835929721,MLA,Soporte Respaldo Apoyo Lumbar Masajeador Asien...,700.00,None,1250,0.44
12,MLA897397255,MLA,Camara Espejo Retrovisor Full Hd + Trasera Est...,2999.00,None,3999,0.250063
17,MLA836882881,MLA,Mini Compresor De Aire Portatil 12v 250 Psi Kl...,1003.00,None,1249,0.196958
...,...,...,...,...,...,...,...
30899,MLA846077762,MLA,Lote De 40 Escallas En Piedras Semipreciosas A...,722.00,None,760,0.05
30903,MLA902021310,MLA,Carbón Bombita Citronella-naranja Sagrada Madr...,363.09,None,399,0.09
30911,MLA904940410,MLA,Pastillas Defumación Activada Sagrada Madre - ...,209.25,None,225,0.07
30919,MLA849741493,MLA,Sahumerios Satya India Californian White Sage ...,126.90,None,135,0.06


**Add category name (specific one for each item)**

In [37]:
df["category_name"] = df.category_id.apply( lambda x: dic_all_categories[x]["name"] if x in dic_all_categories.keys() else None )

In [38]:
def build_categories_from_root(lis):
    res = []
    for x in lis: 
        res.append(x["name"])
    return res

In [39]:
df["categories_from_root"] = df.category_id.apply( lambda x: build_categories_from_root(dic_all_categories[x]["root"]) + [df.category_name] if x in dic_all_categories.keys() else None )

In [40]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.car_dealer_logo,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms,discount,category_name,categories_from_root
0,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,None,ARS,4,19,buy_it_now,gold_special,2040-12-17T04:00:00.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'attribute_group_name': 'Otros', 'name': 'Ma...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[loyalty_discount_eligible, brand_verified, ex...",True,True,1,8911722,http://perfi

## Analize columns

In [41]:
df.sale_price.value_counts()

Series([], Name: sale_price, dtype: int64)

**sale_price column does not provide any useful information**

In [42]:
df["seller.eshop.eshop_rubro"].value_counts()

Series([], Name: seller.eshop.eshop_rubro, dtype: int64)

In [43]:
df["seller.eshop.eshop_rubro.id"].value_counts()

MLA1246      185
MLA5726      125
MLA1182       69
MLA3025       48
MLA1574       30
MLA1384       26
MLA1276       22
MLA1613       22
MLA435173     20
MLA1455       17
MLA6839       15
MLA109278     15
MLA3043       15
MLA109042     14
MLA1631       12
MLA1430       10
MLA3114       10
MLA409810      9
MLA1091        9
MLA3530        8
MLA412445      7
MLA109026      6
MLA6658        6
MLA1263        5
MLA1132        5
MLA1403        5
MLA1575        3
MLA4624        3
MLA1002        3
MLA3959        2
MLA12555       2
MLA1275        2
MLA10934       2
MLA1387        2
MLA5182        1
MLA12554       1
MLA6010        1
MLA1000        1
MLA3724        1
MLA110761      1
MLA1911        1
Name: seller.eshop.eshop_rubro.id, dtype: int64

In [44]:
df[["seller.eshop.eshop_rubro", "id", "seller.eshop.eshop_rubro.id", "seller.eshop.eshop_rubro.name"]].head()

,seller.eshop.eshop_rubro,id,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name
0,NaN,MLA901948080,NaN,NaN
1,NaN,MLA869356478,NaN,NaN
2,NaN,MLA862644604,NaN,NaN
3,NaN,MLA836881964,NaN,NaN
4,NaN,MLA851109741,NaN,NaN


In [45]:
aux = df[["id", "seller.eshop.eshop_rubro", "seller.eshop.eshop_rubro.id", "seller.eshop.eshop_rubro.name"]]
aux[~aux["seller.eshop.eshop_rubro.id"].isna()].head()

,id,seller.eshop.eshop_rubro,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name
870,MLA775628101,NaN,MLA5726,Electrodomésticos y Aires Ac.
1004,MLA603849116,NaN,MLA5726,Electrodomésticos y Aires Ac.
1012,MLA827932245,NaN,MLA1613,Baños
1014,MLA827932039,NaN,MLA1613,Baños
1027,MLA872330266,NaN,MLA5726,Electrodomésticos y Aires Ac.


In [46]:
aux[aux["seller.eshop.eshop_rubro.id"].isna()].head()

,id,seller.eshop.eshop_rubro,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name
0,MLA901948080,NaN,NaN,NaN
1,MLA869356478,NaN,NaN,NaN
2,MLA862644604,NaN,NaN,NaN
3,MLA836881964,NaN,NaN,NaN
4,MLA851109741,NaN,NaN,NaN


**seller.eshop.eshop_rubro is not necessary**

In [47]:
df["seller.eshop.eshop_experience"].value_counts()

0.0       16439
2016.0       92
2012.0       59
1993.0       45
2015.0       44
2007.0       43
2008.0       43
2010.0       28
1984.0       27
1991.0       25
2013.0       23
2011.0       22
2005.0       22
1997.0       21
2014.0       17
2003.0       15
1988.0       12
2006.0       11
1992.0       11
1999.0        9
2009.0        9
1986.0        9
1970.0        8
1995.0        8
1987.0        7
1989.0        7
1994.0        5
1968.0        5
1990.0        4
2001.0        3
1961.0        3
2000.0        2
2004.0        2
1974.0        2
2017.0        1
1998.0        1
1983.0        1
Name: seller.eshop.eshop_experience, dtype: int64

<p style="color:red; font-weight: bold">preguntar: seller.eshop.eshop_experience es un campo llenado por el vendedor? (el típico "since")</p>

In [48]:
df["differential_pricing.id"].value_counts()

33678187.0    1646
35713231.0     378
35713248.0     147
33665186.0     110
35713268.0      67
35713232.0       2
Name: differential_pricing.id, dtype: int64

<p style="color:red; font-weight: bold">preguntar: differential_pricing</p>

In [49]:
df.sale_terms.any()

[{'id': 'PRICE_MODALITY',
  'name': 'Modalidad de precio',
  'value_id': '6178426',
  'value_name': 'Solicitar presupuesto',
  'value_struct': None,
  'values': [{'struct': None,
    'id': '6178426',
    'name': 'Solicitar presupuesto'}]}]

In [50]:
df[df.sale_terms.isna()].shape

(30682, 135)

In [51]:
df[~df.sale_terms.isna()].shape

(243, 135)

In [52]:
df[~df.sale_terms.isna()].head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.car_dealer_logo,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms,discount,category_name,categories_from_root
28121,MLA884308769,MLA,"Instalacion, Colocación Redes Proteccion Balco...",NaN,None,None,1,5,classified,gold_premium,2021-04-15T04:00:33.000Z,None,https://servicio.mercadolibre.com.ar/MLA-88430...,http://http2.mlstatic.com/D_873835-MLA30229546...,873835-MLA30229546338_052019,True,[],None,MLA58261,NaN,MLA-ENCLOSURE_SERVICES,None,"[hirable, dragged_bids_and_visits, good_qualit...",NaN,True,47,50657042,http://perfil.mercadolibre.com.ar/NES6500,2008-06-09T17:24:

In [53]:
df[~df.sale_terms.isna()].root_category_name.value_counts()

Servicios    243
Name: root_category_name, dtype: int64

**sale_terms is present only when root category equals "Servicios"**

In [54]:
df[~df["location.state.id"].isna()].root_category_name.value_counts()

Servicios               1000
Autos, Motos y Otros    1000
Inmuebles               1000
Name: root_category_name, dtype: int64

In [55]:
cols = [
 'address.state_id',
 'address.state_name',
 'address.city_id',
 'address.city_name',
 'seller_address.state.id',
 'seller_address.state.name',
 'seller_address.city.id',
 'seller_address.city.name',
 'location.subneighborhood',
 'location.neighborhood.id',
 'location.neighborhood.name',
 'location.city.id',
 'location.city.name',
 'location.state.id',
 'location.state.name',

]

In [56]:
df[~df["location.state.id"].isna()][cols].head()

,address.state_id,address.state_name,address.city_id,address.city_name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name
5985,TUxBUEdSQWU4ZDkz,Bs.As. G.B.A. Norte,TUxBQ1RJRzk0ZjQw,Tigre,AR-B,Buenos Aires,None,pilar,None,TUxBQlRJRzU0MDla,Tigre,TUxBQ1RJRzk0ZjQw,Tigre,TUxBUEdSQWU4ZDkz,Bs.As. G.B.A. Norte
5986,TUxBUENBUGw3M2E1,Capital Federal,TUxBQ0NBUGZlZG1sYQ,Capital Federal,AR-B,Buenos Aires,TUxBQ1ZJQ2E3MTQz,Vicente López,None,TUxBQkxBQjk1ODJa,La Boca,TUxBQ0NBUGZlZG1sYQ,Capital Federal,TUxBUENBUGw3M2E1,Capital Federal
5987,TUxBUEdSQWU4ZDkz,Bs.As. G.B.A. Norte,TUxBQ1BJTGFyMTIz,Pilar,AR-B,Buenos Aires,None,pilar,None,TVhYQXN0b24gVmlsbGFnZVRVeEJRMUJKVEdGe,Aston Village,TUxBQ1BJTGFyMTIz,Pilar,TUxBUEdSQWU4ZDkz,Bs.As. G.B.A. Norte
5988,TUxBUEdSQXJlMDNm,Bs.As. G.B.A. Sur,TUxBQ0xBTmJjYzE4,Lanús,AR-C,Capital Federal,TUxBQlZJTDcwOTla,Villa Urquiza,None,TUxBQkxBTjM4NDZa,Lanús Oeste,TUxBQ0xBTmJjYzE4,Lanús,TUxBUEdSQXJlMDNm,Bs.As. G.B.A. Sur
5989,TUxBUEdSQWU4ZDkz,Bs.As. G.B.A. Norte,TUxBQ0VTQzQ3YTc0,Escobar,AR-B,Buenos Aires,TUxBQ0VTQzQ3YTc0,Escobar,None,TUxBQkJFTDY3NDda,Belén de Escobar,TUxBQ0VTQzQ3YTc0,Escobar,TUxBUEdSQWU4ZDkz,Bs.As. G.B.A. Norte


In [57]:
df[
    (~df["location.state.id"].isna()) & 
    (df["location.state.id"] != df["address.state_id"])
][cols].head()

,address.state_id,address.state_name,address.city_id,address.city_name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name


In [58]:
df[
    df["seller_address.state.id"] == df["address.state_id"]
][cols].shape

(27926, 15)

In [59]:
df[    
    df["seller_address.state.id"] != df["address.state_id"]
][cols].shape

(2999, 15)

In [60]:
df[
    df["seller_address.state.id"] != df["address.state_id"]
][["address.state_name", "address.state_id", "seller_address.state.name", "seller_address.state.id", "root_category_name"]].drop_duplicates()

,address.state_name,address.state_id,seller_address.state.name,seller_address.state.id,root_category_name
5985,Bs.As. G.B.A. Norte,TUxBUEdSQWU4ZDkz,Buenos Aires,AR-B,"Autos, Motos y Otros"
5986,Capital Federal,TUxBUENBUGw3M2E1,Buenos Aires,AR-B,"Autos, Motos y Otros"
5988,Bs.As. G.B.A. Sur,TUxBUEdSQXJlMDNm,Capital Federal,AR-C,"Autos, Motos y Otros"
5990,Capital Federal,TUxBUENBUGw3M2E1,Capital Federal,AR-C,"Autos, Motos y Otros"
6057,Bs.As. G.B.A. Sur,TUxBUEdSQXJlMDNm,Buenos Aires,AR-B,"Autos, Motos y Otros"
...,...,...,...,...,...
28464,Bs.As. Costa Atlántica,TUxBUENPU2ExMmFkMw,Capital Federal,AR-C,Servicios
28622,Jujuy,TUxBUEpVSnk3YmUz,Jujuy,AR-Y,Servicios
28788,Entre Ríos,TUxBUEVOVHMzNTdm,Entre Ríos,AR-E,Servicios
28830,Córdoba,TUxBUENPUmFkZGIw,Buenos Aires,AR-B,Servicios


In [61]:
df[
    ~df["address.state_id"].isna()
].shape

(30925, 135)

In [62]:
df[
    ~df["seller_address.state.id"].isna()
].shape

(30925, 135)

In [63]:
df.shape

(30925, 135)

In [64]:
df["seller_address.longitude"].value_counts()

    30925
Name: seller_address.longitude, dtype: int64

In [65]:
df[
    df["location.latitude"].eq("")
].shape

c:\users\a309549\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


(0, 135)

In [66]:
df[
    df["seller_address.latitude"].eq("")
].shape

(30925, 135)

In [67]:
df[
    ~df["seller_address.latitude"].isna()
][[
    "address.state_name", "address.state_id", "seller_address.state.name", "seller_address.state.id", 'seller_address.latitude', 'seller_address.longitude',]].head()

,address.state_name,address.state_id,seller_address.state.name,seller_address.state.id,seller_address.latitude,seller_address.longitude
0,Buenos Aires,AR-B,Buenos Aires,AR-B,,
1,Buenos Aires,AR-B,Buenos Aires,AR-B,,
2,Capital Federal,AR-C,Capital Federal,AR-C,,
3,Buenos Aires,AR-B,Buenos Aires,AR-B,,
4,Santa Fe,AR-S,Santa Fe,AR-S,,


<p style="color:red; font-weight: bold">preguntar: address vs seller.address vs location</p>
addess state id parece ser la única necesaria ........... es igual a location si está, y es más específica que la del seller pero está incluida ahí

In [68]:
df["seller.seller_reputation.real_level"].value_counts()

green_platinum    401
yellow            353
green             196
green_gold         78
green_silver       76
orange             50
red                 4
Name: seller.seller_reputation.real_level, dtype: int64

In [69]:
df["seller.seller_reputation.metrics.claims.excluded.real_rate"].value_counts()

0.0000    245
0.0200     61
0.0419     37
0.0162     34
0.0184     30
         ... 
0.1134      1
0.0209      1
0.0265      1
0.0252      1
0.0106      1
Name: seller.seller_reputation.metrics.claims.excluded.real_rate, Length: 153, dtype: int64

In [70]:
df["shipping.logistic_type"].value_counts()

cross_docking    12960
xd_drop_off       5531
fulfillment       4575
not_specified     3480
drop_off          3220
custom             351
default             19
Name: shipping.logistic_type, dtype: int64

In [71]:
df["shipping.mode"].value_counts()

me2              26286
not_specified     4269
custom             351
me1                 19
Name: shipping.mode, dtype: int64

In [72]:
df["seller.seller_reputation.level_id"].value_counts()

5_green          29313
4_light_green      182
3_yellow            70
2_orange            10
1_red                7
Name: seller.seller_reputation.level_id, dtype: int64

In [73]:
df["seller.seller_reputation.metrics.claims.period"].value_counts()

60 days      27178
365 days      3678
60 months       37
Name: seller.seller_reputation.metrics.claims.period, dtype: int64

In [74]:
df.accepts_mercadopago.dtypes

dtype('bool')

In [75]:
df.condition.value_counts()

new              28887
used              1021
not_specified       18
Name: condition, dtype: int64

In [76]:
df.buying_mode.value_counts()

buy_it_now    27925
classified     3000
Name: buying_mode, dtype: int64

In [77]:
df.currency_id.value_counts()

ARS    29777
USD      907
Name: currency_id, dtype: int64

## Choose a subset of columns and persist final df

In [78]:
columns = [
 'installments.quantity',
 'installments.rate',
 'id',
 'site_id',
 'title',
 'price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'buying_mode',
 'listing_type_id',
 'stop_time',
 'condition',
 'accepts_mercadopago',
 'attributes',
 'original_price',
 'category_id',
 'official_store_id',
 'domain_id',
 'catalog_product_id',
 'tags',
 'catalog_listing',
 'seller.id',
 'seller.registration_date',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'seller.tags',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.transactions.ratings.negative',
 'seller.seller_reputation.transactions.ratings.positive',
 'seller.seller_reputation.transactions.ratings.neutral',
 'seller.seller_reputation.transactions.completed',
 'seller.seller_reputation.power_seller_status',
 'seller.seller_reputation.metrics.claims.rate',
 'seller.seller_reputation.metrics.claims.value',
 'seller.seller_reputation.metrics.claims.period',
 'seller.seller_reputation.metrics.delayed_handling_time.rate',
 'seller.seller_reputation.metrics.delayed_handling_time.value',
 'seller.seller_reputation.metrics.delayed_handling_time.period',
 'seller.seller_reputation.metrics.sales.period',
 'seller.seller_reputation.metrics.sales.completed',
 'seller.seller_reputation.metrics.cancellations.rate',
 'seller.seller_reputation.metrics.cancellations.value',
 'seller.seller_reputation.metrics.cancellations.period',
 'seller.seller_reputation.level_id',
 'prices.prices',
 'address.state_id',
 'address.state_name',
 'address.city_id',
 'address.city_name',
 'shipping.free_shipping',
 'shipping.mode',
 'shipping.tags',
 'shipping.logistic_type',
 'shipping.store_pick_up',
 'seller_address.id',
 'seller_address.country.id',
 'seller_address.country.name',
 'seller_address.state.id',
 'seller_address.state.name',
 'seller_address.city.id',
 'seller_address.city.name', 
 'differential_pricing.id',
 'seller.seller_reputation.metrics.claims.excluded.real_rate',
 'seller.seller_reputation.metrics.claims.excluded.real_value',
 'seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate',
 'seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value',
 'seller.seller_reputation.metrics.cancellations.excluded.real_rate',
 'seller.seller_reputation.metrics.cancellations.excluded.real_value',
 'seller.seller_reputation.protection_end_date',
 'seller.seller_reputation.real_level',
 'seller.eshop.eshop_rubro.id',
 'seller.eshop.eshop_rubro.name',
 'seller.eshop.eshop_rubro.category_id',
 'root_category_id',
 'root_category_name',
 'seller_contact.contact',
 'seller_contact.phone',
 'seller_contact.email',
 'seller_contact.webpage',
 'location.neighborhood.id',
 'location.neighborhood.name',
 'location.city.id',
 'location.city.name',
 'location.state.id',
 'location.state.name',
 'location.country.id',
 'location.country.name', 
 'sale_terms',
 'discount',
 'category_name',
 'categories_from_root',

]

In [79]:
columns_discard = [
'stop_time',
'attributes', # marca, condicin, moelo, peso y longitud del paquete
'original_price', # ya tengo el precio posta y el discount, es redundante
# 'official_store_id', # boolean
'domain_id', 	
# 'tags', # mencionar embb
'seller.registration_date',
# IMPORTANT seller.tags # mencionar embb
'seller.seller_reputation.metrics.claims.period',
# por mas de que toma 3 posibles valores, creo que con el rate ya estas ....
'seller.seller_reputation.metrics.claims.value',
# same
# esas ultimas 2 para delayed_handling y sales y cancelations
'seller.seller_reputation.metrics.delayed_handling_time.period',
'seller.seller_reputation.metrics.delayed_handling_time.value',
# IMPORTANT 'seller.seller_reputation.metrics.sales.period',
'seller.seller_reputation.metrics.cancellations.value',
'seller.seller_reputation.metrics.cancellations.period',
'prices',
# (es como atts)
'address.state_id',
'address.city_id',
'address.city_name',
# imposible como categ porque hay un monton .... usaria lat y long
# IMPORTATNT shipping.tags
'seller_address.id',
'seller_address.country.id',
'seller_address.country.name',
'seller_address.state.id',
'seller_address.state.name',
'seller_address.city.id',
'seller_address.city.name',
'differential_pricing.id',
'seller.seller_reputation.metrics.claims.excluded.real_rate',
'seller.seller_reputation.metrics.claims.excluded.real_value',
'seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate',
'seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value',
'seller.seller_reputation.metrics.cancellations.excluded.real_rate',
'seller.seller_reputation.metrics.cancellations.excluded.real_value',
'seller.seller_reputation.protection_end_date',
'seller.seller_reputation.level_id',
'seller.eshop.eshop_rubro.id',
'seller.eshop.eshop_rubro.name',
'seller.eshop.eshop_rubro.category_id',
'seller_contact.contact',
'seller_contact.phone',
'seller_contact.email',
'seller_contact.webpage',
'location.neighborhood.id',
'location.neighborhood.name',
'location.city.id',
'location.city.name',
'location.state.id',
'location.state.name',
'location.country.id',
'location.country.name',
'sale_terms',
'prices.prices'
]

In [80]:
cols = [x for x in columns if x not in columns_discard]
cols

['installments.quantity',
 'installments.rate',
 'id',
 'site_id',
 'title',
 'price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'buying_mode',
 'listing_type_id',
 'condition',
 'accepts_mercadopago',
 'category_id',
 'official_store_id',
 'catalog_product_id',
 'tags',
 'catalog_listing',
 'seller.id',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'seller.tags',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.transactions.ratings.negative',
 'seller.seller_reputation.transactions.ratings.positive',
 'seller.seller_reputation.transactions.ratings.neutral',
 'seller.seller_reputation.transactions.completed',
 'seller.seller_reputation.power_seller_status',
 'seller.seller_reputation.metrics.claims.rate',
 'seller.seller_reputation.metrics.delayed_handling_time.rate',
 'seller.seller_reputation.metrics.delayed_handling_time.value',
 'selle

In [81]:
bool_cols = [
'accepts_mercadopago',
'official_store_id',
    # puedo hacerlo como un boolean de si es oficial o no
'catalog_listing',
'seller.car_dealer',
'seller.real_estate_agency',
'seller.seller_reputation.transactions.period',
    # en todos vale historic
'shipping.free_shipping',
'shipping.store_pick_up',
]

In [82]:
[x for x in bool_cols if x not in cols]

[]

In [83]:
ids = [
'id',
'site_id',
'title',
'category_id',
'seller.id',
'catalog_product_id',
'root_category_id',
'root_category_name',
'category_name',
'categories_from_root',
]

In [84]:
[x for x in ids if x not in cols]

[]

In [85]:
[x for x in cols if 'seller.seller_reputation.metrics.sales' in x]

['seller.seller_reputation.metrics.sales.period',
 'seller.seller_reputation.metrics.sales.completed']

In [86]:
len(columns)

94

## df_final

In [108]:
df_final = df.copy()[cols]  #IMPORTANT antes era columns
df_final.head()

,installments.quantity,installments.rate,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,accepts_mercadopago,category_id,official_store_id,catalog_product_id,tags,catalog_listing,seller.id,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,prices.prices,address.state_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller.seller_reputation.real_level,root_category_id,root_category_name,discount,category_name,categories_from_root
0,12.0,70.17,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,ARS,4,19,buy_it_now,gold_special,new,True,MLA6867,NaN,MLA14983192,"[loyalty_discount_eligible, brand_verified, ex...",True,8911722,False,False,"[normal, user_info_verified, eshop, credits_pr...",410.0,30.0,historic,0.01,0.99,0.00,380.0,silver,0.0000,0.0000,0.0,60 days,60 days,63.0,0.0000,0.0,60 days,"[{'id': '17', 'type': 'standard', 'conditions'...",Buenos Aires,True,me2,"[self_service_in, mandatory_free_shipping]",drop_off,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Estéreos,"[Accesorios para Vehículos, Audio para Vehícul..."
1,12.0,70.17,MLA869356478,MLA,Neumático Pirelli Formula Energy 185/60 R14 82h,10250.0,ARS,71,1472,buy_it_now,gold_special,new,True,MLA22195,143,MLA15000133,"[brand_verified, good_quality_picture, good_qu...",True,116026826,False,False,"[brand, user_info_verified, large_seller, esho...",22558.0,973.0,historic,0.01,0.96,0.03,21585.0,platinum,0.0041,0.0558,166.0,60 days,60 days,3648.0,0.0044,17.0,60 days,"[{'id': '24', 'type': 'standard', 'conditions'...",Buenos Aires,True,me2,[mandatory_free_shipping],cross_docking,True,NaN,MLA5725,Accesorios para Vehículos,NaN,Neumáticos de Auto y Camioneta,"[Accesorios para Vehículos, Neumáticos, [Estér..."
2,12.0,70.17,MLA862644604,MLA,Casco Para Moto Integral Halcon H57 Negro Tall...,2289.5,ARS,41,418,buy_it_now,gold_special,new,True,MLA4712,NaN,MLA11386900,"[good_quality_thumbnail, loyalty_discount_elig...",True,19876329,False,False,"[normal, user_info_verified, credits_priority_...",188730.0,14338.0,historic,0.03,0.94,0.03,174392.0,platinum,0.0166,0.0643,1503.0,60 days,60 days,24591.0,0.0132,351.0,60 days,"[{'id': '6', 'type': 'standard', 'conditions':...",Capital Federal,False,me2,"[self_service_in, fs_threshold_mla_change_feb2...",cross_docking,False,NaN,MLA5725,Accesorios para Vehículos,0.05,Cascos,"[Accesorios para Vehículos, Acc. para Motos y ..."
3,12.0,70.17,MLA836881964,MLA,Compresor De Aire Alto Impacto Portátil 250 Ps...,1899.0,ARS,500,500,buy_it_now,gold_special,new,True,MLA370797,1825,MLA16452983,"[dragged_bids_and_visits, good_quality_picture...",NaN,417972986,False,False,"[brand, user_info_verified, large_seller, msho...",364825.0,16464.0,historic,0.03,0.96,0.01,348361.0,platinum,0.0067,0.0000,0.0,60 days,60 days,63441.0,0.0009,62.0,60 days,[{'id': 'c98a86c0-79ec-47a5-8ea4-f605f4c0b369'...,Buenos Aires,False,me2,"[fulfillment, fs_threshold_mla_change_feb2021]",fulfillment,False,NaN,MLA5725,Accesorios para Vehículos,0.321543,Mini Compresores,"[Accesorios para Vehícul

In [109]:
df_final.dtypes

installments.quantity                                            float64
installments.rate                                                float64
id                                                                object
site_id                                                           object
title                                                             object
price                                                            float64
currency_id                                                       object
available_quantity                                                 int64
sold_quantity                                                      int64
buying_mode                                                       object
listing_type_id                                                   object
condition                                                         object
accepts_mercadopago                                                 bool
category_id                                        

### Change types

official_store_id boolean

In [110]:
df_final['official_store_id'].isna().any()

True

In [113]:
df_final['official_store_id']

0         NaN
1         143
2         NaN
3        1825
4         NaN
         ... 
30920     NaN
30921     NaN
30922     NaN
30923     NaN
30924     NaN
Name: official_store_id, Length: 30925, dtype: object

In [114]:
df_final['official_store_id'] = ~df_final['official_store_id'].isna() 


In [115]:
df_final['official_store_id'].head()

0    False
1     True
2    False
3     True
4    False
Name: official_store_id, dtype: bool

In [122]:
df_final.rename(columns={'official_store_id': 'is_official_store'}, inplace=True)

In [129]:
df_final['is_official_store'].head()

0    False
1     True
2    False
3     True
4    False
Name: is_official_store, dtype: bool

In [134]:
df_final.columns

Index(['installments.quantity', 'installments.rate', 'id', 'site_id', 'title',
       'price', 'currency_id', 'available_quantity', 'sold_quantity',
       'buying_mode', 'listing_type_id', 'condition', 'accepts_mercadopago',
       'category_id', 'is_official_store', 'catalog_product_id', 'tags',
       'catalog_listing', 'seller.id', 'seller.car_dealer',
       'seller.real_estate_agency', 'seller.tags',
       'seller.seller_reputation.transactions.total',
       'seller.seller_reputation.transactions.canceled',
       'seller.seller_reputation.transactions.period',
       'seller.seller_reputation.transactions.ratings.negative',
       'seller.seller_reputation.transactions.ratings.positive',
       'seller.seller_reputation.transactions.ratings.neutral',
       'seller.seller_reputation.transactions.completed',
       'seller.seller_reputation.power_seller_status',
       'seller.seller_reputation.metrics.claims.rate',
       'seller.seller_reputation.metrics.delayed_handling_time

bools to int

In [146]:
bool_cols = [
'accepts_mercadopago',
'is_official_store',
    # puedo hacerlo como un boolean de si es oficial o no
'catalog_listing',
'seller.car_dealer',
'seller.real_estate_agency',
'shipping.free_shipping',
'shipping.store_pick_up',
]

In [144]:
df_final = df_final.infer_objects() 

In [145]:
df_final['is_official_store']

0        False
1         True
2        False
3         True
4        False
         ...  
30920    False
30921    False
30922    False
30923    False
30924    False
Name: is_official_store, Length: 30925, dtype: bool

In [181]:
df_final[bool_cols] = df_final[bool_cols].fillna(0).astype(int)
df_final.dtypes 

installments.quantity                                             float64
installments.rate                                                 float64
id                                                                 object
site_id                                                            object
title                                                              object
price                                                             float64
currency_id                                                        object
available_quantity                                               category
sold_quantity                                                    category
buying_mode                                                        object
listing_type_id                                                    object
condition                                                          object
accepts_mercadopago                                                 int32
category_id                           

### available quantity

In [182]:
df_final.available_quantity.isna().any()

False

In [153]:
df_final.available_quantity.value_counts().sort_index()

1        17840
2            2
3            5
4            2
5            4
         ...  
3667         1
5000       893
7239         1
7639         1
50000      327
Name: available_quantity, Length: 83, dtype: int64

In [154]:
bins = [0, 50, 100, 150, 200, 250, 500, 5000, 50000, 99999]

In [156]:
labels = [
    1, 50, 100, 150, 200, 250, 500, 5000, 50000,
]

In [157]:
df_final["available_quantity_q"] = pd.cut(df_final['available_quantity'], bins=bins, right=False, labels=labels)  

In [158]:
df_final[[
    "available_quantity",
    "available_quantity_q",
]].head(20)

,available_quantity,available_quantity_q
0,4,1
1,71,50
2,41,1
3,500,500
4,50,50
5,50,50
6,50,50
7,500,500
8,150,150
9,1,1


In [166]:
df_final.available_quantity_q.value_counts().sort_index()

AttributeError: 'DataFrame' object has no attribute 'available_quantity_q'

In [161]:
df_final.available_quantity = df_final.available_quantity_q

In [165]:
df_final.drop('available_quantity_q', axis = 1, inplace=True)

### Target

In [170]:
df_final.sold_quantity.value_counts().sort_index()

0        3157
1         287
2         241
3         235
4         203
         ... 
7056        1
8508        1
9005        1
10132       1
50000       9
Name: sold_quantity, Length: 95, dtype: int64

<p style="color:red; font-weight: bold">Hay valores que no tienen el value esperado (que representa cada categoria) asi que trabajamos eso. </p>

In [171]:
bins = [0, 1, 2, 3, 4, 5, 25, 50, 100, 150, 200, 250, 500, 5000, 50000, 500000]

In [172]:
labels = [
    0, 1, 2, 3, 4, 5, 25, 50, 100, 150, 200, 250, 500, 5000, 50000,
]

In [173]:
df_final["sold_quantity_q"] = pd.cut(df_final['sold_quantity'], bins=bins, right=False, labels=labels)  

In [174]:
df_final[[
    "sold_quantity",
    "sold_quantity_q",
]].head(20)

,sold_quantity,sold_quantity_q
0,19,5
1,1472,500
2,418,250
3,500,500
4,500,500
5,500,500
6,500,500
7,5000,5000
8,500,500
9,500,500


In [176]:
df_final.sold_quantity_q.value_counts().sort_index()

0        3157
1         287
2         241
3         235
4         203
5        2342
25       1822
50       2792
100      2068
150      1756
200      1438
250      4706
500      9151
5000      718
50000       9
Name: sold_quantity_q, dtype: int64

In [177]:
df_final.sold_quantity = df_final.sold_quantity_q

In [178]:
df_final.drop('sold_quantity_q', axis = 1, inplace=True)

### Export final df

In [179]:
data_path = "Data/"
file_name = data_path + "data.csv"

In [180]:
df_final.to_csv(file_name, sep='\t', encoding='utf-8', index=False)